<a href="https://colab.research.google.com/github/TimurTimur02/AIHomework/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%BF%D0%BE%D1%82%D0%B5%D0%BC%D0%B5%E2%80%9C%D0%A4%D1%80%D0%B5%D0%B9%D0%BC%D0%B2%D0%BE%D1%80%D0%BA%D0%B8Python%D0%B4%D0%BB%D1%8FML(CatBoost%2Cscikitlearn).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import numpy as np
import sklearn as sk
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2

предметы = ['Математика', 'Русский язык', 'Информатика', 'Химия', 'Биология', 'История']

количество_записей = 2000
средний_балл = np.random.randint(0, 100, количество_записей)
оценка_лабораторной = np.random.choice(['Удовлетворительно', 'Хорошо', 'Отлично'], количество_записей)
время_решения_лабораторной = np.random.randint(0, 60, количество_записей)

данные = pd.DataFrame({
    'Предмет': np.random.choice(предметы, количество_записей),
    'Средний балл': средний_балл,
    'Оценка за лабораторную': оценка_лабораторной,
    'Время решения лабораторной': время_решения_лабораторной
})

данные.to_csv('data.csv', index=False)

данные = pd.read_csv('data.csv')

X = данные.drop('Оценка за лабораторную', axis=1)
y = данные['Оценка за лабораторную']

label_encoder = LabelEncoder()
X['Предмет'] = label_encoder.fit_transform(X['Предмет'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

k_best_chi2 = SelectKBest(score_func=chi2, k=3)
k_best_chi2.fit(X, y)

selected_features_chi2 = X.columns[k_best_chi2.get_support()]

best_features_model_chi2 = RandomForestClassifier(random_state=42)
best_features_model_chi2.fit(X[selected_features_chi2], y)

joblib.dump(best_features_model_chi2, 'best_features_model_chi2.onnx')

loaded_model_chi2 = joblib.load('best_features_model_chi2.onnx')

X_train, X_test, y_train, y_test = train_test_split(X[selected_features_chi2], y, test_size=0.2, random_state=42)

y_pred_chi2 = loaded_model_chi2.predict(X_test)

accuracy_chi2 = accuracy_score(y_test, y_pred_chi2)

loaded_model = joblib.load('best_features_model_chi2.onnx')

subject_scores = [55, 14, 72, 11, 22, 3]

def predict_student_grade(subject_scores):
    input_data = np.array(subject_scores).reshape(1, -1)
    input_data_selected = input_data[:, :3]

    predicted_grade = best_features_model_chi2.predict(input_data_selected)

    return predicted_grade[0]

predicted_grade = predict_student_grade(subject_scores)

print("Прогноз оценки:", predicted_grade + "\n")

print("Точность модели на тестовой выборке с использованием критерия хи-квадрат:", accuracy_chi2)

print("Лучшие 3 признака:", selected_features_chi2 + "\n")

print("Размер обучающей выборки (X_train, y_train):", X_train.shape, y_train.shape)

print("Размер тестовой выборки (X_test, y_test):", X_test.shape, y_test.shape)

print(данные.head())


Прогноз оценки: Хорошо

Точность модели на тестовой выборке с использованием критерия хи-квадрат: 0.99
Лучшие 3 признака: Index(['Предмет\n', 'Средний балл\n', 'Время решения лабораторной\n'], dtype='object')
Размер обучающей выборки (X_train, y_train): (1600, 3) (1600,)
Размер тестовой выборки (X_test, y_test): (400, 3) (400,)
       Предмет  Средний балл Оценка за лабораторную  \
0   Математика            55      Удовлетворительно   
1  Информатика            42                Отлично   
2  Информатика            43                Отлично   
3   Математика            26                Отлично   
4     Биология            54                 Хорошо   

   Время решения лабораторной  
0                          55  
1                          19  
2                          39  
3                          24  
4                           5  


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
